<a href="https://colab.research.google.com/github/citizenyves/Advanced_NLP_with_PyTorch/blob/main/6_BertWordPieceTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 6.5 MB 5.3 MB/s 


In [ ]:
import torch
from tokenizers import BertWordPieceTokenizer

In [ ]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla K80
cuda


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/wanted_preonboarding/week3/3-1

/content/drive/MyDrive/Colab Notebooks/wanted_preonboarding/week3/3-1


In [ ]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
!pip install gdown
!gdown https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
!unzip processed.zip

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG 

Archive:  processed.zip
replace processed/space-correct.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/processed_korquad.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/processed_ratings_train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/processed_ratings_test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/processed_wiki_ko.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/processed_ratings.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/corrected_ratings_corpus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace processed/soyword.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
repl

In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

My current directory : /content/drive/MyDrive/Colab Notebooks/wanted_preonboarding/week3/3-1


### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [ ]:
# processed_wiki_ko.txt 파일 불러오기

docs = pd.read_csv(f"{_DATA_DIR}/processed_wiki_ko.txt", header=None, sep='\t')

In [ ]:
docs

,0
0,"제임스 얼 ""지미"" 카터 주니어 (, 1924년 10월 1일 ~ )는 민주당 출신 ..."
1,"수학 (數學, Mathematics) 은 양, 구조, 공간, 변화 등의 개념을 다루..."
2,"수학에서 상수 란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와..."
3,"The Reader.jpg|섬네일|250px|장오노레 프라고나르 작 ""책 읽는 소녀..."
4,"이 문서는 나라 목록 이며, 전 세계 206개 나라 의 각 현황과 주권 승인 정보를..."
...,...
311232,하인리히 보그트 (1890년 10월 5일 - 1968년 1월 23일)는 독일의 천문...
311233,김경호 (1995년 7월 31일 ~ )는 KBO 리그 두산 베어스의 외야수이다. 2...
311234,"1584 년과 1729 년 사이에 출판 된 ""히브리어 라틴 신성한 성경"" 10 판 ..."
311235,김태권 (金兌權)은 대한민국의 만화가 겸 저술가이다. 《김태권의 십자군 이야기》를 ...


In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [ ]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # 악센트가 있는 character의 악센트를 제거하려면? (ex. é → e)
    lowercase=False, # 한국어는 대소문자가 없는데 소문자 변환이 필요한지?
)

In [ ]:
# train
# files: 'processed_wiki_ko.txt'
# vocab_size: 30,000
# min_frequency: 2
# special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# limit_alphabet: 1,000
# wordpieces_prefix: '##'

tokenizer.train(
    files = ['processed/processed_wiki_ko.txt'], #data path
    vocab_size = 30000, #만들고자 하는 vocab의 size, 보통 '32000' 정도가 좋다고 알려져 있다
    min_frequency = 2, #merge를 수행할 최소 빈도수, 5로 설정 시 5회 이상 등장한 pair만 수행한다
    show_progress = True, #학습 진행과정 show
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"], #Tokenizer에 추가하고 싶은 special token 지정
    limit_alphabet = 1000, #merge 수행 전 initial tokens이 유지되는 숫자 제한
    wordpieces_prefix = '##', 
)

In [ ]:
tokenizer.save_model(".", "wordpiece")

['./wordpiece-vocab.txt']

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [ ]:
_CUR_DIR

'/content/drive/MyDrive/Colab Notebooks/wanted_preonboarding/week3/3-1'

In [ ]:
tokenizer = BertWordPieceTokenizer(
    vocab = './wordpiece-vocab.txt', #"wordpiece-vocab.txt 경로",
    lowercase = False,
    strip_accents = False,
)

In [ ]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

30000

In [ ]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
tokenized = tokenizer.encode(text)

token_ids = tokenized.ids
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
tokens = tokenized.tokens
print(tokens)

[2, 7864, 20862, 16, 509, 3371, 5566, 2778, 5757, 16, 3]
['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']


In [ ]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text1 = "과제 개킹받네ㅠ"
text2 = "어쩔티비 저쩔티비 슈슈슈-슉-슈슉"

tokenized1 = tokenizer.encode(text1)
tokenized2 = tokenizer.encode(text2)

tokens1 = tokenized1.tokens
tokens2 = tokenized2.tokens

print(tokens1)
print(tokens2)

['[CLS]', '과제', '[UNK]', '[SEP]']
['[CLS]', '[UNK]', '[UNK]', '슈', '##슈', '##슈', '-', '[UNK]', '-', '[UNK]', '[SEP]']


In [ ]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "어쩔티비"

tokenized = tokenizer.encode(text)

unknown_token_ids = tokenized.ids # 토큰 id
unknown_tokens = tokenized.tokens # 토큰
print(unknown_token_ids)
print(unknown_tokens)

[2, 1, 3]
['[CLS]', '[UNK]', '[SEP]']


### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [ ]:
# 원래 문장: "안녕하세요. 버트를 사용한 모델입니다."
tokenizer.decode(token_ids)

'안녕하세요. 버트를 사용한 모델입니다.'

In [ ]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)

''